In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
# import train_test_split 
from sklearn.model_selection import train_test_split
from os import listdir
from methods import ARTICLES_DIR,remove_stop_words
from typing import List,Dict
import matplotlib.pyplot as plt


TODO:
 1. load all the documents
 2. vectorize each document, 1 for word existing in the document, 0 for not existing
 3. cluster the documents

לטעון את כל המסמכים

להפריד כל מסמך לפי רווח רגיל

לעשות אוסף של המילים

ולעבור כל מסמך ומסמך, האם מילה קיימת או לא.

בלי stemming ובלי הורדת מילות טפל

In [2]:
#TODO: change the directories and remove files to get to 50 total articles.
paths=[ARTICLES_DIR,
       f"{ARTICLES_DIR}/other groups/5 Technology for cultural heritage/text",
       f"{ARTICLES_DIR}/other groups/12 Visitors guide archeological sites/text files",
       f"{ARTICLES_DIR}/other groups/14 Information presentation museum/txt"
    ]

In [3]:
# כתוב פונקציה שמקבלת מחרוזת, מחזירה מערך של מחרוזות של המילים במחרוזת המסופקת
# מוסיפה את הרשימה שחוזרת לתוך אובייקט מסוג set

def split_str(document:str)->List[str]:
    return document.split()


In [4]:
dic:Dict[int,Dict[str,bool]]={}
set_of_words=set()

In [5]:
for path in paths:
    arts=listdir(path)
    for art in arts:
        if art=='.DS_Store': continue
        try:
            with open(f"{path}/{art}",'r',encoding="UTF-8") as file:
                set_of_words.update(split_str(file.read()))
        except:
            print("Error reading file",path,art)


Error reading file ./articels other groups
Error reading file ./articels/other groups/5 Technology for cultural heritage/text pdftotext3.zip


In [6]:
with open('stop_words_english.txt','r',encoding="UTF-8") as file:
    stop_words=set(file.read().split())

In [7]:
set_of_stop_words=set(stop_words)

In [8]:
# len(set_of_words)

In [9]:
# len(set_of_words-set_of_stop_words)

In [10]:
set_of_words-=set_of_stop_words

In [11]:
len(set_of_words)
dic.clear()

In [12]:
# dic:Dict[int,Dict[str,bool]]={}
id=1
for path in paths:
    arts=listdir(path)
    for art in arts:
        if art=='.DS_Store': continue
        try:
            with open(f"{path}/{art}",'r',encoding="UTF-8") as file:
                temp:Dict[str,bool]={}
                content=set(remove_stop_words(file.read()).split())
                
                # content=remove_stop_words(file.read())
                for word in set_of_words:
                    temp[word]=1 if word in content else 0
                    # temp[word]=content.count(word)
                dic[id]=temp
                id+=1
        except:
            print("Error reading file",path,art)


Error reading file ./articels other groups
Error reading file ./articels/other groups/5 Technology for cultural heritage/text pdftotext3.zip


In [13]:
id,len(dic)

(199, 198)

In [14]:
df=pd.DataFrame.from_dict(dic,orient='index')

In [15]:
from pickle import dump


In [16]:
len(df),len(df.columns)

(198, 149856)

In [18]:
df["category"]=df.index.map(lambda x: int(x//50)+1)

In [19]:
with open("df after category - model vectors for classification.pcl","wb+") as file:
    dump(df,file)

In [22]:
df.to_csv("df after category - model vectors for classification.csv",index=False)

In [ ]:
df.head()

In [ ]:
df["category"].value_counts()

In [ ]:
# df : collection of vectors
X=df.drop(["category"],axis=1)
y=df["category"]

X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.10)